 This notebook will be mainly used for the IBM Applied Data Science Capstone Project

In [2]:
import pandas as pd
import numpy as np
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


# IBM Applied Data Science Capstone Project

In [1]:
#first I install geopy & folium
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0     conda-forge

The following packages will be UPDATED:

    geopy:         1.11.0-py36_0 conda-forge --> 1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executi

In [114]:
#then, I import the libraries needed
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import lxml.html as lh

print('Libraries imported.')

Libraries imported.


Now I create a function to get the coordinates of a given neighborhood of CABA (short for the City of Buenos Aires)

In [115]:
def GetLoc(name, city = ', CABA'):

    address = name + city

    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

    return latitude, longitude

Now I retrieve the data, convert it to a dataframe a get coordinates for each neighborhood

In [116]:
pricesDF = pd.read_csv('labs/DA0101EN/Prices_m2_data.csv')
pricesDF = pricesDF.reindex(columns=[*pricesDF.columns.tolist(), 'Latitude', 'Longitude'])

for i in pricesDF.index:
    LL = GetLoc(pricesDF['Neighborhood'][i])
    pricesDF['Latitude'][i] = LL[0]
    pricesDF['Longitude'][i] = LL[1]
    
pricesDF.head(7)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Neighborhood,Avg Price per square meter,Latitude,Longitude
0,Villa Lugano,1417,-34.674153,-58.477689
1,Villa Riachuelo,1940,-34.691206,-58.471450
2,Boca,2034,-34.633510,-58.359042
3,Velez Sarsfield,2064,-34.631196,-58.492556
4,Nueva Pompeya,2072,-34.652630,-58.414710
5,Parque Patricios,2104,-34.637482,-58.401239
6,Versalles,2104,-34.628928,-58.523299


Now I get the coordinates of CABA to center the map I will create, and create such map with the neighborhoods retrieved from the CSV file

In [117]:
CABA = GetLoc('CABA')
CABA

(-34.6075616, -58.437076)

In [118]:
latitude = CABA[0]
longitude = CABA[1]

map_caba = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(pricesDF['Latitude'], pricesDF['Longitude'], pricesDF['Neighborhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_caba)  
    
map_caba

Now lets add the Foursquare data

In [119]:

CLIENT_ID = 'Z15JZA0CDQLNUGIUFAIX4IPUZKZ024ZURVNQVWP5UHFHDXJ3' 
CLIENT_SECRET = 'VJXBHHIAJ3Z3RKNAWSAOP1GZDEHSBPHNN4SLF0P4JWXX4WGE' 
VERSION = '20180605' 

LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

caba_venues = getNearbyVenues(names=pricesDF['Neighborhood'],
                                   latitudes=pricesDF['Latitude'],
                                   longitudes=pricesDF['Longitude']
                                  )
caba_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Villa Lugano,-34.674153,-58.477689,Restaurante Piuke,-34.673089,-58.478277,Argentinian Restaurant
1,Villa Lugano,-34.674153,-58.477689,La Moderna,-34.676075,-58.476685,Dessert Shop
2,Villa Lugano,-34.674153,-58.477689,Nicolo,-34.671339,-58.478458,Ice Cream Shop
3,Villa Lugano,-34.674153,-58.477689,A Pesar De Todo,-34.676958,-58.475185,Bar
4,Villa Lugano,-34.674153,-58.477689,La Perla,-34.677730,-58.478185,Bakery


In [120]:
# one hot encoding
caba_onehot = pd.get_dummies(caba_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
caba_onehot['Neighborhood'] = caba_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [caba_onehot.columns[-1]] + list(caba_onehot.columns[:-1])
caba_onehot = caba_onehot[fixed_columns]

caba_grouped = caba_onehot.groupby('Neighborhood').mean().reset_index()

def return_least_common_venues(row, num_last_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=True)
    
    return row_categories_sorted.index.values[0:num_last_venues]

num_last_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of last venues
columns = ['Neighborhood']
for ind in np.arange(num_last_venues):
    try:
        columns.append('{}{} Least Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Least Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = caba_grouped['Neighborhood']

for ind in np.arange(caba_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_least_common_venues(caba_grouped.iloc[ind, :], num_last_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue
0,Agronomia,American Restaurant,Multiplex,Museum,Music Venue,Nightclub
1,Almagro,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
2,Balvanera,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
3,Barracas,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
4,Belgrano,American Restaurant,Motel,Motorcycle Shop,Multiplex,Museum


Now I will cluster neighborhoods by average price per square meter and analyze each cluster to determine which price range it contains.

In [121]:
# set number of clusters
kclusters = 5

pricesDF_clustering = pricesDF.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pricesDF_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1,
       1, 1, 2], dtype=int32)

In [122]:
pricesDFlabels = pricesDF
pricesDFlabels['Cluster Labels'] = kmeans.labels_
pricesDFlabels.head()

,Neighborhood,Avg Price per square meter,Latitude,Longitude,Cluster Labels
0,Villa Lugano,1417,-34.674153,-58.477689,0
1,Villa Riachuelo,1940,-34.691206,-58.471450,0
2,Boca,2034,-34.633510,-58.359042,0
3,Velez Sarsfield,2064,-34.631196,-58.492556,0
4,Nueva Pompeya,2072,-34.652630,-58.414710,0


In [123]:
pricesDFlabels.loc[pricesDFlabels['Cluster Labels'] == 0, pricesDFlabels.columns[[1] + list(range(5, pricesDFlabels.shape[1]))]]

,Avg Price per square meter
0,1417
1,1940
2,2034
3,2064
4,2072
5,2104
6,2104
7,2141
8,2146
9,2270


In [124]:
pricesDFlabels.loc[pricesDFlabels['Cluster Labels'] == 1, pricesDFlabels.columns[[1] + list(range(5, pricesDFlabels.shape[1]))]]

,Avg Price per square meter
39,3449
40,3494
41,3511
42,3651
43,3973
44,4028
45,4152


In [125]:
pricesDFlabels.loc[pricesDFlabels['Cluster Labels'] == 2, pricesDFlabels.columns[[1] + list(range(5, pricesDFlabels.shape[1]))]]

,Avg Price per square meter
46,6729


In [126]:
pricesDFlabels.loc[pricesDFlabels['Cluster Labels'] == 3, pricesDFlabels.columns[[1] + list(range(5, pricesDFlabels.shape[1]))]]

,Avg Price per square meter
27,2803
28,2859
29,2892
30,2906
31,2974
32,3015
33,3028
34,3031
35,3068
36,3074


In [127]:
pricesDFlabels.loc[pricesDFlabels['Cluster Labels'] == 4, pricesDFlabels.columns[[1] + list(range(5, pricesDFlabels.shape[1]))]]

,Avg Price per square meter
12,2378
13,2393
14,2404
15,2418
16,2474
17,2510
18,2521
19,2521
20,2524
21,2525


Now I will create two functions that will help me customize PopUps in the map, getPrice to assign to each cluster a price area, and popUp to get the information I want into the popup feature of the map.

In [128]:
def getPrice(cluster):
    if cluster == 0:
        return 'Very Cheap Area'
    elif cluster == 1:
        return 'Expensive Area'
    elif cluster == 2:
        return 'Very Expensive Area'
    elif cluster == 3:
        return 'Regular Price Area'
    else:
        return 'Cheap Area'

def popUp (cluster, neighborhood):
    #for i in neighborhoods_venues_sorted.index:
    i = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighborhood']==neighborhood].index[0]
    least_common_venues= neighborhoods_venues_sorted.iloc[i, 1:6]
    return neighborhood + ': a ' + getPrice(cluster) + '; Least Common Venues: ' + least_common_venues[0] + ', ' + least_common_venues[1] + ', ' + least_common_venues[2] + ', ' + least_common_venues[3]+ ' and ' + least_common_venues[4]

popUp(2,'Almagro')

'Almagro: a Very Expensive Area; Least Common Venues: American Restaurant, Museum, Music Venue, Nightclub and Noodle House'

Now I create the map:

In [129]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pricesDFlabels['Latitude'], pricesDFlabels['Longitude'], pricesDFlabels['Neighborhood'], pricesDFlabels['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=popUp(cluster,poi),
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The entire data is provided as a table below:

In [130]:
labels = pricesDFlabels
for i in labels.index:
    Label = getPrice(labels['Cluster Labels'][i])
    labels['Cluster Labels'][i] = Label

AllData = pd.merge(labels, neighborhoods_venues_sorted, left_on = 'Neighborhood', right_on = 'Neighborhood')
AllData

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Neighborhood,Avg Price per square meter,Latitude,Longitude,Cluster Labels,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue
0,Villa Lugano,1417,-34.674153,-58.477689,Very Cheap Area,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
1,Villa Riachuelo,1940,-34.691206,-58.471450,Very Cheap Area,American Restaurant,Multiplex,Museum,Music Venue,Nightclub
2,Boca,2034,-34.633510,-58.359042,Very Cheap Area,American Restaurant,Multiplex,Museum,Music Venue,Nightclub
3,Velez Sarsfield,2064,-34.631196,-58.492556,Very Cheap Area,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
4,Nueva Pompeya,2072,-34.652630,-58.414710,Very Cheap Area,American Restaurant,Motorcycle Shop,Multiplex,Museum,Music Venue
5,Parque Patricios,2104,-34.637482,-58.401239,Very Cheap Area,American Restaurant,Multiplex,Museum,Music Venue,Nightclub
6,Versalles,2104,-34.628928,-58.523299,Very Cheap Area,American Restaurant,Motorcycle Shop,Multiplex,Museum,Music Venue
7,Floresta,2141,-34.628105,-58.483791,Very Cheap Area,American Restaurant,Multiplex,Museum,Music Venue,Nightclub
8,Constitucion,2146,-34.628258,-58.380570,Very Cheap Area,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
9,Balvanera,2270,-34.609215,-58.403140,Very Cheap Area,American Restaurant,Museum,Music Venue,Nightclub,Noodle House
